# This notebook is provided if you'd like to use the object detection evaluation module (objdeteval.py) to evaluate the results output from the scrdet_parallel module that were saved and formatted in the myproject_cv_api.ipynb Jupyter Notebook. **Note that because this example uses the very few detections and groundtruth annotations provided with the SCRDet MYPROJECT PARALLEL repository, the curves will look a little funky. **

In [1]:
import numpy as np

# Evaluation inputs
det_file = 'my_results.txt'
gt_file = '../Sample_NITF/groundtruth_for_eval.txt'
iou_thresh = 0.2
score_thresh = [0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
## MyClasses ##
class_list = ['class1', 'class2', 'class3']

In [2]:
from objdeteval import Eval

## The usage for the Eval function is as follows:

**Eval(gt_file,
     det_file,
     iou_thresh,
     class_list,
     score_thresh=array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),)**

***Evaluates a detection file against a groundtruth file for object-oriented quadrilaters.
Returns precision, recall, F1-Score, True-Positives, False-Positives, False-Negatives,
and a confusion matrix***

**Usage: p,r,f1,tp,fp,fn,confm = Eval(gt_file, det_file, iou_thresh, score_thresh, class_list)**

**gt_file** : str; path to single groundtruth text file containing ALL annotations for ALL images
          you are evaluating for. Each line of the groundtruth text-file is formatted
          as follows:
              image_id class_name x0 y0 x1 y1 x2 y2 x3 y3
              
**det_file** : str; path to single detection text file containing ALL detection info for ALL images
          you tested on. Each line of the detection text-file is formatted
          as follows:
              class_name image_id confidence_score x0 y0 x1 y1 x2 y2 x3 y3 score_class_0:n
              
              Note: Including the score for each class (score_class_0:n) is not required

**iou_thresh** : float; The acceptable Intersection-Over-Union for a detection to be considered a 
             True-Positive

**score_thresh** : iterable : a list or array of confidence scores to evaluate against. E.g., 
               score_thresh = np.arange(0.0,1.1,0.1)
               
**class_list** : list; An list of strings corresponding to the classes in the groundtruth (does
             not include the 'background' class). E.g., class_list=['class1','class2','class3']

In [3]:
[p,r,f1,tp,fp,fn,confm] = Eval(gt_file=gt_file,det_file=det_file,iou_thresh=iou_thresh,score_thresh=score_thresh,class_list=class_list)

converted txt files to arrays

done, computing p r and f1
done


## The outputs will have a shape equal to ***(len(score_thresh),len(class_list))***
## The confusion matrix will have a shape equal to ***(len(class_list)+1,len(class_list),len(score_thresh))***, where the first dimension is one longer because it includes the "background" class

In [4]:
p.shape,r.shape,f1.shape,tp.shape,fp.shape,fn.shape,confm.shape

((11, 3), (11, 3), (11, 3), (11, 3), (11, 3), (11, 3), (4, 3, 11))

In [5]:
# find the average metrics for each score threshold
p_mean = np.nanmean(p,axis=1)
r_mean = np.nanmean(r,axis=1)
f1_mean = np.nanmean(f1,axis=1)

## The below cells are used to plot the results

In [6]:
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.patheffects as pe

In [1]:
##############################
## Plot the average metrics ##
##############################

fig,ax = plt.subplots(1,2,figsize=(20,20))

ax[0].plot(r_mean,p_mean,linewidth=4,c='r',path_effects=[pe.Stroke(linewidth=6, foreground='k'), pe.Normal()])
ax[0].plot(r_mean,p_mean,'o',c='k',mew=5)
ax[0].set_ylabel('Precision',size=20)
ax[0].set_xlabel('Recall',size=20)
ax[0].axis('square');
ax[0].set_title('PR Curve (AP = '+str(np.round(np.nanmean(p_mean),decimals=3))+')',size=20);
ax[0].minorticks_on()
ax[0].grid()
ax[0].grid(which='minor', linestyle=':', linewidth='0.1', color='black')
ax[0].set_xlim(-0.01,1.01);
ax[0].set_ylim(-0.01,1.01);
ax[0].tick_params(labelsize=22)

ax[1].plot(score_thresh,p_mean,linewidth=4,c='r',path_effects=[pe.Stroke(linewidth=6, foreground='k'), pe.Normal()])
ax[1].plot(score_thresh,r_mean,linewidth=4,c='b',path_effects=[pe.Stroke(linewidth=6, foreground='k'), pe.Normal()])
ax[1].plot(score_thresh,f1_mean,linewidth=4,c='m',path_effects=[pe.Stroke(linewidth=6, foreground='k'), pe.Normal()])
ax[1].legend(['Precision','Recall','F1'],fontsize=20,loc='lower left')
ax[1].set_xlabel('Confidence Score',size=20);
ax[1].set_ylabel('Value',size=20);
ax[1].axis('square');
ax[1].set_title('Average Metrics vs Confindence',size=20);
ax[1].minorticks_on()
ax[1].grid()
ax[1].grid(which='minor', linestyle=':', linewidth='0.1', color='black')
ax[1].set_xlim(-0.01,1.01);
ax[1].set_ylim(-0.01,1.01);
ax[1].tick_params(labelsize=22)

NameError: name 'plt' is not defined

In [1]:
#######################################
## Plot the class-specific PR curves ##
#######################################

fig0,ax0 = plt.subplots(figsize=(10,10))

w = cm.get_cmap('gist_rainbow')
linsty = ['solid','dotted','dashed']*20
for cl in np.arange(0,len(class_list)):
    ax0.plot(r[:,cl],p[:,cl],color=w(cl/(len(class_list)-1)),path_effects=[pe.Stroke(linewidth=6, foreground='k'), pe.Normal()],linestyle = linsty[cl],linewidth=4)
ax0.legend(class_list,loc='right',bbox_to_anchor=(1.35, 0.5),prop={'size': 14},fontsize=40)
ax0.set_xlabel('Recall',size=20);
ax0.set_ylabel('Precision',size=20);
ax0.axis('square');
ax0.set_title('PR Class Breakdown',size=20);
ax0.minorticks_on()
ax0.grid()
ax0.grid(which='minor', linestyle=':', linewidth='0.1', color='black')
ax0.set_xlim(-0.01,1.01);
ax0.set_ylim(-0.01,1.01);
ax0.tick_params(labelsize=22)

NameError: name 'plt' is not defined

In [2]:
#####################################################################
## Plot the class-specific F1, Precision, and Recall vs Confidence ##
#####################################################################

fig0,ax0 = plt.subplots(1,3,figsize=(25,25))

w = cm.get_cmap('gist_rainbow')
linsty = ['solid','dotted','dashed']*20
for cl in np.arange(0,len(class_list)):
    ax0[0].plot(score_thresh,f1[:,cl],color=w(cl/(len(class_list)-1)),path_effects=[pe.Stroke(linewidth=6, foreground='k'), pe.Normal()],linestyle = linsty[cl],linewidth=4)
ax0[0].set_xlabel('Confidence Score',size=20);
ax0[0].set_ylabel('Value',size=20);
ax0[0].axis('square');
ax0[0].set_title('F1 Class Breakdown',size=20);
ax0[0].minorticks_on()
ax0[0].grid()
ax0[0].grid(which='minor', linestyle=':', linewidth='0.1', color='black')
ax0[0].set_xlim(-0.01,1.01);
ax0[0].set_ylim(-0.01,1.01);
ax0[0].tick_params(labelsize=22)


for cl in np.arange(0,len(class_list)):
    ax0[1].plot(score_thresh,r[:,cl],color=w(cl/(len(class_list)-1)),path_effects=[pe.Stroke(linewidth=6, foreground='k'), pe.Normal()],linestyle = linsty[cl],linewidth=4)
ax0[1].set_xlabel('Confidence Score',size=20);
ax0[1].axis('square');
ax0[1].set_title('Recall Class Breakdown',size=20);
ax0[1].minorticks_on()
ax0[1].grid()
ax0[1].grid(which='minor', linestyle=':', linewidth='0.1', color='black')
ax0[1].set_xlim(-0.01,1.01);
ax0[1].set_ylim(-0.01,1.01);
ax0[1].tick_params(labelsize=22)


for cl in np.arange(0,len(class_list)):
    ax0[2].plot(score_thresh,p[:,cl],color=w(cl/(len(class_list)-1)),path_effects=[pe.Stroke(linewidth=6, foreground='k'), pe.Normal()],linestyle = linsty[cl],linewidth=4)
ax0[2].legend(class_list,loc='right',bbox_to_anchor=(1.42, 0.5),prop={'size': 14},fontsize=40)
ax0[2].set_xlabel('Confidence Score',size=20);
ax0[2].axis('square');
ax0[2].set_title('Precision Class Breakdown',size=20);
ax0[2].minorticks_on()
ax0[2].grid()
ax0[2].grid(which='minor', linestyle=':', linewidth='0.1', color='black')
ax0[2].set_xlim(-0.01,1.01);
ax0[2].set_ylim(-0.01,1.01);
ax0[2].tick_params(labelsize=22)

NameError: name 'plt' is not defined

In [3]:
######################################################
## Plot the confusion matrix where confidence = 0.5 ##
######################################################
import pandas as pd
pd.DataFrame(confm[:,:,5].astype(int),columns=class_list,index=['Background']+class_list)

NameError: name 'confm' is not defined